In [26]:
from utils import process_ns3_csv
filename = '20241119_st-100_N-50_BH-20_SF-0_L-15_uni-1'
input_file = './dataset/' + filename + '/MonitorSnifferRx.csv'

output_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx.csv'
log_file = './dataset/' + filename + '/log1119_BH0.1_uniform_15#%15seed_12345.txt'
process_ns3_csv(input_file, output_file, log_file)

FileNotFoundError: [Errno 2] No such file or directory: './dataset/20241119_st-100_N-50_BH-20_SF-0_L-15_uni-1/log1119_BH0.1_uniform_15#%15seed_12345.txt'

In [ ]:
from utils import load_data
import numpy as np
import models 

In [ ]:
data_file = './dataset/' + filename + '/ProcessedMonitorSnifferRx.csv'

In [ ]:
np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)
# print(processed_data[0].edge_attr)
edge_feature_size = processed_data[0].edge_attr.shape[1]
nodes_feature_size = processed_data[0].x.shape[1]


In [ ]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(processed_data)
# print(adj[12])
# print(T[12])

In [ ]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[-1].edge_index.numpy().size)

In [ ]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
# print(processed_data[0].x[:, 0:23].numpy())
# print(processed_data[15].y.numpy())
# print(processed_data[15].edge_index)
# print(processed_data[12].edge_weights)
# print(processed_data[12].x)

In [27]:
train_model = models.HypergraphModel(nodes_feature_size, edge_feature_size)
print(train_model)

HypergraphModel(
  (layer1): HypergraphConvLayer(
    (conv): HypergraphConv(23, 8)
  )
  (edge_conv): GraphConvolution()
  (class_classifier): Sequential(
    (0): Linear(in_features=8, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=2, bias=True)
  )
  (norm1): CustomBatchNorm()
  (norm2): CustomBatchNorm()
)


In [28]:
import time
import torch.optim as optim
from utils import accuracy, auc, precision, recall, f1_score
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
train_end_index = 4
val_index = 6
def train(epoch, save_path):
    t = time.time()
    train_model.train()
    optimizer.zero_grad()
    # print(processed_data[0].x[0:23].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(train_end_index):
        print(i)
        output = train_model(processed_data[i].x, processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].edge_attr, adj[i], T[i])
        # print(output)
        labels = processed_data[i].y
        loss_train = criteria(output, labels)
        # print(output[:, 1])
        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()

        
        train_model.eval()
        output = train_model(processed_data[val_index].x, processed_data[val_index].edge_index, 
                                 processed_data[val_index].edge_weights, processed_data[val_index].edge_attr, adj[val_index], T[val_index])
        # print(output)
        labels = processed_data[val_index].y
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        auc_val = auc(output, labels)
        per_val = precision(output, labels)
        recall_val = recall(output, labels)
        f1_val = f1_score(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
            'auc_val: {:.4f}'.format(auc_val),
            'per_val: {:.4f}'.format(per_val),
            'recall_val: {:.4f}'.format(recall_val),
            'f1_val: {:.4f}'.format(f1_val),
              'time: {:.4f}s'.format(time.time() - t))
    # 每个 epoch 结束后保存模型
    torch.save(train_model.state_dict(), save_path)
    print(f'Model saved to {save_path}')

    return loss_val.item()


In [29]:
def test():
    train_model.eval()
    testIndex = 8
    output = train_model(processed_data[testIndex].x, processed_data[testIndex].edge_index, 
                             processed_data[testIndex].edge_weights, processed_data[testIndex].edge_attr, adj[testIndex], T[testIndex])
    labels = processed_data[group_num-1].y
    # print(output)
    # print(labels)
    loss_test = criteria(output, labels)
    acc_test = acc_measure(output, labels)
    auc_test = auc(output, labels)
    per_test = precision(output, labels)
    recall_test = recall(output, labels)
    f1_test = f1_score(output, labels)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
         'auc_test: {:.4f}'.format(auc_test),
            'per_test: {:.4f}'.format(per_test),
            'recall_test: {:.4f}'.format(recall_test),
            'f1_test: {:.4f}'.format(f1_test))
    return acc_test.item()

In [30]:
import os
# torch.autograd.set_detect_anomaly(True)
num_epochs = 1000
early_stopping = 30
val_watch = []
t_total = time.time()
current_time = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
save_path = './models/' + current_time + '/' + current_time + '.pth'
os.makedirs(os.path.dirname(save_path), exist_ok=True)  # 创建目录，如果目录已存在不抛出错误
for epoch in range(num_epochs):
    val = train(epoch, save_path)  # 训练并获得结果
    if val is None:  # 检查是否返回了None值
        print(f"Warning: train(epoch) returned None for epoch {epoch}. Skipping this iteration.")
        continue
    
    val_watch.append(val)  # 将结果添加到val_watch
    test()
    # if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
    #     print("Early stopping...")
    #     break
    
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")

0
loss_train: 1.1214 acc_train: 0.5600 loss_val: 82.6834 acc_val: 0.1000 auc_val: 0.5000 per_val: 0.1000 recall_val: 1.0000 f1_val: 0.1818 time: 0.0196s
Model saved to ./models/2024-11-22_20-30-16/2024-11-22_20-30-16.pth
Test set results: loss= 90.1877 accuracy= 0.1000 auc_test: 0.5000 per_test: 0.1000 recall_test: 1.0000 f1_test: 0.1818
0
loss_train: 1.0905 acc_train: 0.5600 loss_val: 38.5907 acc_val: 0.1000 auc_val: 0.5111 per_val: 0.1000 recall_val: 1.0000 f1_val: 0.1818 time: 0.0213s
Model saved to ./models/2024-11-22_20-30-16/2024-11-22_20-30-16.pth
Test set results: loss= 39.5278 accuracy= 0.1000 auc_test: 0.5556 per_test: 0.1000 recall_test: 1.0000 f1_test: 0.1818
0
loss_train: 1.0609 acc_train: 0.5600 loss_val: 21.5806 acc_val: 0.1000 auc_val: 0.5111 per_val: 0.1000 recall_val: 1.0000 f1_val: 0.1818 time: 0.0165s
Model saved to ./models/2024-11-22_20-30-16/2024-11-22_20-30-16.pth
Test set results: loss= 20.9885 accuracy= 0.1200 auc_test: 0.6267 per_test: 0.1020 recall_test: 1.0


KeyboardInterrupt



In [ ]:
# 加载模型

train_model.load_state_dict(torch.load(save_path, weights_only=True))
train_model.eval()  # 切换到评估模式